In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("data.csv")
g_features = []
g_target = []
for i in range(len(df["diameter"])):
    g_features.append([df["diameter"][i],df["area"][i],df["radius"][i],[df["cataract"][i],df["other"][i]])
    if df["status"][i] == 1:
        g_target.append(1)
    else:
        g_target.append(0)
print(len(g_features))
g_features = np.array(g_features)
g_target = np.array(g_target)

In [ ]:
#cross-validation and plot ROC curves
print(__doc__)

import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import StratifiedKFold
#from sklearn.model_selection import StratifiedShuffleSplit

# #############################################################################
# Data IO and generation

# Import some data to play with
X = g_features
y = g_target
X, y = X[y != 2], y[y != 2]
n_samples, n_features = X.shape


# Add noisy features
random_state = np.random.RandomState(0)


X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=random_state)
# #############################################################################
# Classification and ROC analysis


# Run classifier with cross-validation and plot ROC curves

cv = StratifiedKFold(n_splits=5,shuffle=True)
#cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2)
classifier = svm.SVC(kernel='linear', probability=True)
history = []
accuracy = []
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
fig, ax = plt.subplots()
for i, (train, val) in enumerate(cv.split(X_train, y_train)):
    
    X_train, X_val = X[train], X[val]
    y_train, y_val= y[train], y[val]
    classifier.fit(X_train, y_train)
    predict = classifier.predict(X_val)
    print(predict)
    history.append(classification_report(y_val,predict))
    clf_probs = classifier.predict_proba(X_val)
    acc = accuracy_score(y_val, predict)
    accuracy.append(acc)
    loss = log_loss(y_val, clf_probs)
    print(f"accuracy_score : {acc}")
    print(f"log_loss : {loss}\n")
    viz = plot_roc_curve(classifier, X[val], y[val],
                         name='ROC fold {}'.format(i),
                         alpha=0.3, lw=1, ax=ax)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
    
ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='b',
        label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
       title="Receiver operating characteristic example")
ax.legend(loc="lower right")
plt.savefig('svm_normal_graph.pdf')
plt.show()
accuracy = np.array(accuracy)
print(f"accuracy : {np.mean(accuracy)}")

for i in range(len(history)):
    print(history[i])

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(classifier, X_test, y_test)
plt.savefig('svm_normal_confusion_matrix.pdf')

In [ ]:
#cross-validation and plot ROC curves
print(__doc__)

import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
#from sklearn.model_selection import StratifiedShuffleSplit

# #############################################################################
# Data IO and generation

# Import some data to play with
X = g_features
y = g_target
X, y = X[y != 2], y[y != 2]
n_samples, n_features = X.shape


# Add noisy features
random_state = np.random.RandomState(0)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=random_state)
# #############################################################################
# Classification and ROC analysis


# Run classifier with cross-validation and plot ROC curves

cv = StratifiedKFold(n_splits=5,shuffle=True)
#cv = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=random_state)
classifier = KNeighborsClassifier(n_neighbors=4)
history = []
accuracy = []
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
fig, ax = plt.subplots()
for i, (train, val) in enumerate(cv.split(X_train, y_train)):
    
    X_train, X_val = X[train], X[val]
    y_train, y_val= y[train], y[val]
    classifier.fit(X_train, y_train)
    predict = classifier.predict(X_val)
    print(predict)
    history.append(classification_report(y_val,predict))
    clf_probs = classifier.predict_proba(X_val)
    acc = accuracy_score(y_val, predict)
    accuracy.append(acc)
    loss = log_loss(y_val, clf_probs)
    print(f"accuracy_score : {acc}")
    print(f"log_loss : {loss}\n")
    viz = plot_roc_curve(classifier, X[val], y[val],
                         name='ROC fold {}'.format(i),
                         alpha=0.3, lw=1, ax=ax)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
    
ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='b',
        label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
       title="Receiver operating characteristic example")
ax.legend(loc="lower right")
plt.savefig('knn_normal_graph.pdf')
plt.show()
accuracy = np.array(accuracy)
print(f"accuracy : {np.mean(accuracy)}")

for i in range(len(history)):
    print(history[i])

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(classifier, X_test, y_test)
plt.savefig('knn_normal_confusion_matrix.pdf')